# Fetch model from Hugging face hub, tag and store to mlflow UC registry

Model used: [OpenGVL/InternVL3-8B](https://huggingface.co/OpenGVLab/InternVL3-8B)


In [0]:
%pip install --upgrade mlflow-skinny timm transformers


%restart_python

In [0]:
import mlflow
from mlflow import MlflowClient


mlflow.set_registry_uri("databricks-uc")
client  = MlflowClient()

In [0]:
CATALOG = "amine_elhelou" # Change This/Point to an existing catalog
SCHEMA = "ray_gtm_examples" # Point to an existing schema
VOLUME = "fashion-images"

In [0]:
import torch
import gc


# Empty GPU memory
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# Run garbage collection
gc.collect()

InternVL3-8B

In [0]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModel


hf_model_path = "OpenGVLab/InternVL3-8B"
hf_model = AutoModel.from_pretrained(
  hf_model_path,
  torch_dtype=torch.bfloat16,
  low_cpu_mem_usage=True,
  trust_remote_code=True,
  use_flash_attn=True,
)
# .eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(hf_model_path, trust_remote_code=True)

In [0]:
model_weights_path = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/{hf_model_path.replace('/', '_')}"
hf_model.save_pretrained(f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/{model_weights_path}")

In [0]:
import torch
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec, Array
from transformers import pipeline


torch_dtype=torch.bfloat16
MODEL_NAME = "InternVL3-8B"

# Define the input schema
input_schema = Schema([
    ColSpec("string", "prompt"),  # User query
    ColSpec(Array("double"), "pixel_values"),  # Image tensor
])

# Define the output schema (generated text response)
output_schema = Schema([
    ColSpec("string", "response")
])

# Create the model signature
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

In [0]:
internvl_pipe = pipeline(
    "image-text-to-text",
    model=hf_model,
    tokenizer=tokenizer,
    torch_dtype=torch_dtype,
)

In [0]:
# Log the pipeline
with mlflow.start_run(run_name="intern-vl-log-pipeline"):
    model_info = mlflow.transformers.log_model(
        transformers_model=internvl_pipe,
        name="ivl_pipeline",
        signature=signature,
        registered_model_name=f"{CATALOG}.{SCHEMA}.{MODEL_NAME}",
    )

In [0]:
client.set_registered_model_alias(
  name=f"{CATALOG}.{SCHEMA}.{MODEL_NAME}",
  version=model_info.registered_model_version,
  alias="production",
)